# 📊 ESG Composition Risk Intelligence

 

### Import Libraries & Setup 

In [12]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import random
import warnings

warnings.filterwarnings('ignore') # Suppress warnings for clarity

In [13]:
#  Set reproducibility seed 
np.random.seed(42) 

### Define Company & Industry Metadata

#### Each company is mapped to:
   - Industry classification
   - City headquarters
   - ESG materiality notes (for some sectors)

In [14]:
companies_data = {
    'HDFC Bank': {'industry': 'Finance', 'city': 'Mumbai'},
    'Reliance Industries': {'industry': 'Energy', 'city': 'Mumbai'},
    'Infosys': {'industry': 'Technology', 'city': 'Bengaluru'},
    'Tata Motors': {'industry': 'Automotive', 'city': 'Pune'},
    'Apollo Hospitals': {'industry': 'Healthcare', 'city': 'Chennai'},
    'Asian Paints': {'industry': 'Manufacturing', 'city': 'Mumbai'},
    'L&T': {'industry': 'Infrastructure', 'city': 'Mumbai'},
    'Wipro': {'industry': 'Technology', 'city': 'Bengaluru'},
    'Maruti Suzuki': {'industry': 'Automotive', 'city': 'Gurgaon'},
    'DMart (Avenue Supermarts)': {'industry': 'Retail', 'city': 'Mumbai'},
    'Power Grid Corp': {'industry': 'Energy', 'city': 'Gurgaon'},
    'Godrej Consumer Products': {'industry': 'FMCG', 'city': 'Mumbai'},
    'Bharti Airtel': {'industry': 'Telecom', 'city': 'Delhi'},
    'Nestle India': {'industry': 'FMCG', 'city': 'Gurgaon'},
    'Adani Ports': {'industry': 'Infrastructure', 'city': 'Ahmedabad'},
    'SBI': {'industry': 'Finance', 'city': 'Mumbai'}, 
    'TCS': {'industry': 'Technology', 'city': 'Mumbai'},
    'JSW Steel': {'industry': 'Manufacturing', 'city': 'Mumbai'}, 
    'Cipla': {'industry': 'Pharmaceuticals', 'city': 'Mumbai'},
    'IndusInd Bank': {'industry': 'Finance', 'city': 'Mumbai'},
    'Vedanta Ltd': {'industry': 'Metals & Mining', 'city': 'Mumbai'}, 
    'Aditya Birla Fashion': {'industry': 'Retail', 'city': 'Mumbai'},
    'Dr. Reddy\'s Labs': {'industry': 'Pharmaceuticals', 'city': 'Hyderabad'},
    'HCL Tech': {'industry': 'Technology', 'city': 'Noida'},
    'Indian Oil Corp': {'industry': 'Energy', 'city': 'Delhi'}, 
    'Bajaj Finance': {'industry': 'Finance', 'city': 'Pune'},
    'United Spirits': {'industry': 'FMCG', 'city': 'Bengaluru'},
    'Grasim Industries': {'industry': 'Diversified', 'city': 'Mumbai'}, 
    'Zydus Lifesciences': {'industry': 'Pharmaceuticals', 'city': 'Ahmedabad'},
    'Siemens India': {'industry': 'Industrial Conglomerate', 'city': 'Mumbai'}
}

#  Define Geospatial Coordinates & Climate Risk Index

# Each city has:
#  - Latitude & Longitude (for mapping in Power BI)
#   - Climate Risk Index (captures coastal/heatwave/flood risk)

city_coordinates = {
    'Mumbai': (19.0760, 72.8777),
    'Bengaluru': (12.9716, 77.5946),
    'Chennai': (13.0827, 80.2707),
    'Pune': (18.5204, 73.8567),
    'Gurgaon': (28.4595, 77.0266),
    'Delhi': (28.7041, 77.1025),
    'Ahmedabad': (23.0225, 72.5714),
    'Hyderabad': (17.3850, 78.4867),
    'Noida': (28.5355, 77.3910)
}

# Simulated Climate Risk Index based on city/region (higher is riskier)
city_climate_risk = {
    'Mumbai': 7.5, 
    'Bengaluru': 5.0,
    'Chennai': 8.0, 
    'Pune': 4.0,
    'Gurgaon': 6.5,
    'Delhi': 7.0,
    'Ahmedabad': 7.0, 
    'Hyderabad': 5.5,
    'Noida': 6.8 
}

# Generate ESG Metrics for Each Company

# Categories:
#   - Environmental (Scope 1/2 GHG, Water, Waste, Renewables)
#   - Social (Leadership diversity, Turnover, CSR, Grievances)
#   - Governance (Board Independence, Audit, Corruption)
#   - Financial (Liquidity, Leverage, Growth)
#   - Sentiment & Controversy (Simulated media coverage)
#   - Regulatory Readiness (BRSR & TCFD flags)
data = []

for company_name, details in companies_data.items():
    industry = details['industry']
    city_hq = details['city']
    lat, lon = city_coordinates[city_hq]
    simulated_climate_risk = city_climate_risk[city_hq] + np.random.uniform(-1.0, 1.0) 

    # --- Environmental Metrics (BRSR Core aligned) ---
    scope1_ghg = np.random.uniform(5000, 50000) if industry not in ['Technology', 'Finance', 'FMCG'] else np.random.uniform(100, 1000)
    scope2_ghg = np.random.uniform(10000, 100000) if industry not in ['Technology', 'Finance', 'FMCG'] else np.random.uniform(200, 2000)
    water_intensity = np.random.uniform(0.5, 5.0) 
    waste_diversion_rate = np.random.uniform(30, 95) 
    renewable_energy_perc = np.random.uniform(5, 70) 

    # --- Social Metrics (BRSR Core aligned) ---
    female_leadership_ratio = np.random.uniform(10, 40) 
    employee_turnover_rate = np.random.uniform(5, 30) 
    community_investment_perc_pbt = np.random.uniform(0.5, 3.0) 
    grievances_resolved_perc = np.random.uniform(70, 100) 

    # --- Governance Metrics (BRSR Core aligned) ---
    board_independence = np.random.uniform(50, 90) 
    audit_committee_independence = np.random.uniform(80, 100) 
    corruption_incidents = np.random.randint(0, 3) 
   
    # --- Financial Metrics ---
    debt_equity = np.random.uniform(0.5, 2.5)
    current_ratio = np.random.uniform(0.7, 3.5)
    revenue_growth = np.random.uniform(-5, 25)

    # --- Injecting higher risk for some specific companies ---
    if company_name in ['SBI', 'JSW Steel', 'Vedanta Ltd', 'Indian Oil Corp']:
        # Financial stress
        debt_equity = np.random.uniform(2.0, 4.0) 
        current_ratio = np.random.uniform(0.3, 1.0) 
        revenue_growth = np.random.uniform(-10, 5) 
        
        # ESG/Governance issues
        corruption_incidents = np.random.randint(1, 4) 
        board_independence = np.random.uniform(30, 60) 
        base_sentiment = np.random.uniform(-0.9, -0.2) 
        controversy_flag = 1 
        chosen_controversy = random.choice(["environmental fine", "governance lapse", "labour unrest", "scandal"])
        controversy_detail = chosen_controversy
    else:
        # --- News Sentiment & Controversy  ---
        base_sentiment = np.random.uniform(-0.8, 0.8) 
        controversy_keywords = ["pollution fine", "labour dispute", "governance scandal", "regulatory action", "fraud investigation", "greenwashing claims", "data breach"]
        controversy_flag = 0 
        controversy_detail = ""

        # Simulate controversy occurrence for others
        if np.random.rand() < 0.20: 
            chosen_controversy = random.choice(controversy_keywords)
            base_sentiment -= np.random.uniform(0.3, 0.7) 
            controversy_flag = 1
            controversy_detail = chosen_controversy

    news_sentiment = round(max(-1, min(1, base_sentiment)), 2) # Clip to -1 to 1 range


    # --- Regulatory Readiness (BRSR/TCFD alignment) ---
    tcfd_flag = 1 if np.random.rand() < 0.75 else 0 
    brsr_flag = 1 if np.random.rand() < 0.85 else 0 

    # --- Simulate Missing Data (for robust handling demonstration) ---
    if np.random.rand() < 0.1: 
        scope1_ghg = np.nan
    if np.random.rand() < 0.05: 
        female_leadership_ratio = np.nan
    if np.random.rand() < 0.05: 
        board_independence = np.nan

    data.append([
        company_name, industry, city_hq, round(lat, 4), round(lon, 4), round(simulated_climate_risk, 2),
        round(scope1_ghg, 2), round(scope2_ghg, 2), round(water_intensity, 2), round(waste_diversion_rate, 2), round(renewable_energy_perc, 2),
        round(female_leadership_ratio, 2), round(employee_turnover_rate, 2), round(community_investment_perc_pbt, 2), round(grievances_resolved_perc, 2),
        round(board_independence, 2), round(audit_committee_independence, 2), corruption_incidents,
        round(debt_equity, 2), round(current_ratio, 2), round(revenue_growth, 2),
        news_sentiment, controversy_flag, controversy_detail,
        tcfd_flag, brsr_flag
    ])

# Define Column Schema & Export Data
# ------------------------------------------------
# Final dataset = ESG + Financial + Sentiment + Regulatory
# Output: ESG_Enhanced_Raw_Data.csv

columns = [
    'Company', 'Industry', 'City_HQ', 'Latitude', 'Longitude', 'Simulated_Climate_Risk_Index',
    'Scope1_GHG_Emissions', 'Scope2_GHG_Emissions', 'Water_Consumption_Intensity', 'Waste_Diversion_Rate', 'Renewable_Energy_Percentage',
    'Female_Leadership_Ratio', 'Employee_Turnover_Rate', 'Community_Investment_PBT_Percentage', 'Grievances_Resolved_Percentage',
    'Board_Independence_Percentage', 'Audit_Committee_Independence_Percentage', 'Corruption_Incidents',
    'Debt_To_Equity', 'Current_Ratio', 'Revenue_Growth_Percentage',
    'News_Sentiment', 'Controversy_Flag', 'Controversy_Detail',
    'TCFD_Disclosure_Flag', 'SEBI_BRSR_Flag'
]

df_raw = pd.DataFrame(data, columns=columns)
df_raw.to_csv('ESG_Enhanced_Raw_Data.csv', index=False)

print("ESG Enhanced Raw Data (30 companies) generated and saved as 'ESG_Enhanced_Raw_Data.csv'")

ESG Enhanced Raw Data (30 companies) generated and saved as 'ESG_Enhanced_Raw_Data.csv'


In [15]:
def run_advanced_esg_risk_engine(input_csv='ESG_Enhanced_Raw_Data.csv', output_csv='ESG_Risk_Scored_Output.csv'):
    """
    Processes raw ESG and financial data, handles missing values, calculates
    normalized and weighted risk scores, and provides risk explanations.
    """
    try:
        df = pd.read_csv(input_csv)
    except FileNotFoundError:
        print(f"Error: Input file '{input_csv}' not found. Please ensure 'generate_esg_dataset.py' was run first.")
        return

    print(f"Processing data from '{input_csv}'...")

    # --- 1. Handle Missing Data ---
    # For simplicity, we'll impute NaNs with the median of their respective columns.
    # In a real scenario, more sophisticated methods (e.g., K-NN imputation, regression)
    # or domain-specific strategies would be used.
    for col in ['Scope1_GHG_Emissions', 'Scope2_GHG_Emissions', 'Water_Consumption_Intensity', 'Waste_Diversion_Rate',
                'Female_Leadership_Ratio', 'Employee_Turnover_Rate', 'Community_Investment_PBT_Percentage',
                'Grievances_Resolved_Percentage', 'Board_Independence_Percentage',
                'Audit_Committee_Independence_Percentage', 'Debt_To_Equity', 'Current_Ratio', 'Revenue_Growth_Percentage']:
        if col in df.columns and df[col].isnull().any():
            median_val = df[col].median()
            df[col].fillna(median_val, inplace=True)
            print(f"  - Imputed missing values in '{col}' with median: {median_val:.2f}")

    # --- 2. Normalize Metrics (0-1 scale) ---
    # We'll use MinMaxScaler. For metrics where a HIGHER value is BETTER, we invert the scaled score (1 - scaled_value)
    # to ensure that a higher normalized score always implies higher risk.

    scaler = MinMaxScaler()
    # Environmental
    df['Scope1_GHG_Scaled'] = scaler.fit_transform(df[['Scope1_GHG_Emissions']])
    df['Scope2_GHG_Scaled'] = scaler.fit_transform(df[['Scope2_GHG_Emissions']])
    df['Water_Intensity_Scaled'] = scaler.fit_transform(df[['Water_Consumption_Intensity']])
    df['Waste_Diversion_Scaled'] = 1 - scaler.fit_transform(df[['Waste_Diversion_Rate']]) 
    df['Renewable_Energy_Scaled'] = 1 - scaler.fit_transform(df[['Renewable_Energy_Percentage']]) 
    # Social 
    df['Employee_Turnover_Scaled'] = scaler.fit_transform(df[['Employee_Turnover_Rate']])
    df['Community_Investment_Scaled'] = 1 - scaler.fit_transform(df[['Community_Investment_PBT_Percentage']]) 
    df['Female_Leadership_Scaled'] = 1 - scaler.fit_transform(df[['Female_Leadership_Ratio']]) 
    df['Grievances_Resolved_Scaled'] = 1 - scaler.fit_transform(df[['Grievances_Resolved_Percentage']]) 
    df['News_Sentiment_Risk_Scaled'] = (df['News_Sentiment'] * -1 + 1) / 2 

    # Governance 
    df['Board_Independence_Scaled'] = 1 - scaler.fit_transform(df[['Board_Independence_Percentage']]) 
    df['Audit_Committee_Scaled'] = 1 - scaler.fit_transform(df[['Audit_Committee_Independence_Percentage']]) 
    df['Corruption_Incidents_Scaled'] = scaler.fit_transform(df[['Corruption_Incidents']]) 

    # Financial 
    df['Debt_Equity_Scaled'] = scaler.fit_transform(df[['Debt_To_Equity']])
    df['Current_Ratio_Scaled'] = 1 - scaler.fit_transform(df[['Current_Ratio']]) 
    df['Revenue_Growth_Scaled'] = 1 - scaler.fit_transform(df[['Revenue_Growth_Percentage']]) 

    # Simulated Climate Risk (already on a risk scale, but good to ensure 0-1)
    df['Climate_Risk_Scaled'] = scaler.fit_transform(df[['Simulated_Climate_Risk_Index']])

    # --- 3. Materiality-Based Scoring & Pillar Calculation ---
    # Define weights based on industry materiality (conceptual for this simulation)
    # These weights would be refined based on in-depth industry analysis and regulatory focus.
    industry_weights = {
        'Finance': {
            'E': {'Scope1_GHG_Scaled': 0.1, 'Scope2_GHG_Scaled': 0.1, 'Water_Intensity_Scaled': 0.05, 'Waste_Diversion_Scaled': 0.05, 'Renewable_Energy_Scaled': 0.1},
            'S': {'Female_Leadership_Scaled': 0.2, 'Employee_Turnover_Scaled': 0.1, 'Community_Investment_Scaled': 0.1, 'Grievances_Resolved_Scaled': 0.1, 'News_Sentiment_Risk_Scaled': 0.5}, # High sentiment impact for banks
            'G': {'Board_Independence_Scaled': 0.4, 'Audit_Committee_Scaled': 0.3, 'Corruption_Incidents_Scaled': 0.3},
            'Financial': {'Debt_Equity_Scaled': 0.35, 'Current_Ratio_Scaled': 0.35, 'Revenue_Growth_Scaled': 0.3},
            'Compliance': {'TCFD_Disclosure_Flag': 0.5, 'SEBI_BRSR_Flag': 0.5}, 
            'Climate': {'Climate_Risk_Scaled': 1.0}
        },
        'Energy': {
            'E': {'Scope1_GHG_Scaled': 0.4, 'Scope2_GHG_Scaled': 0.3, 'Water_Intensity_Scaled': 0.1, 'Waste_Diversion_Scaled': 0.1, 'Renewable_Energy_Scaled': 0.1},
            'S': {'Female_Leadership_Scaled': 0.1, 'Employee_Turnover_Scaled': 0.2, 'Community_Investment_Scaled': 0.3, 'Grievances_Resolved_Scaled': 0.2, 'News_Sentiment_Risk_Scaled': 0.2},
            'G': {'Board_Independence_Scaled': 0.4, 'Audit_Committee_Scaled': 0.3, 'Corruption_Incidents_Scaled': 0.3},
            'Financial': {'Debt_Equity_Scaled': 0.4, 'Current_Ratio_Scaled': 0.3, 'Revenue_Growth_Scaled': 0.3},
            'Compliance': {'TCFD_Disclosure_Flag': 0.6, 'SEBI_BRSR_Flag': 0.4},
            'Climate': {'Climate_Risk_Scaled': 1.0} 
        },
        'Technology': {
            'E': {'Scope1_GHG_Scaled': 0.2, 'Scope2_GHG_Scaled': 0.2, 'Water_Intensity_Scaled': 0.1, 'Waste_Diversion_Scaled': 0.1, 'Renewable_Energy_Scaled': 0.4},
            'S': {'Female_Leadership_Scaled': 0.4, 'Employee_Turnover_Scaled': 0.3, 'Community_Investment_Scaled': 0.1, 'Grievances_Resolved_Scaled': 0.2, 'News_Sentiment_Risk_Scaled': 0.2},
            'G': {'Board_Independence_Scaled': 0.4, 'Audit_Committee_Scaled': 0.3, 'Corruption_Incidents_Scaled': 0.3},
            'Financial': {'Debt_Equity_Scaled': 0.3, 'Current_Ratio_Scaled': 0.3, 'Revenue_Growth_Scaled': 0.4},
            'Compliance': {'TCFD_Disclosure_Flag': 0.5, 'SEBI_BRSR_Flag': 0.5},
            'Climate': {'Climate_Risk_Scaled': 1.0}
        },
        'Healthcare': {
            'E': {'Scope1_GHG_Scaled': 0.2, 'Scope2_GHG_Scaled': 0.2, 'Water_Intensity_Scaled': 0.3, 'Waste_Diversion_Scaled': 0.3, 'Renewable_Energy_Scaled': 0.0}, # Waste more critical
            'S': {'Female_Leadership_Scaled': 0.2, 'Employee_Turnover_Scaled': 0.2, 'Community_Investment_Scaled': 0.1, 'Grievances_Resolved_Scaled': 0.5, 'News_Sentiment_Risk_Scaled': 0.0}, # Patient/employee safety more critical than sentiment
            'G': {'Board_Independence_Scaled': 0.4, 'Audit_Committee_Scaled': 0.3, 'Corruption_Incidents_Scaled': 0.3},
            'Financial': {'Debt_Equity_Scaled': 0.3, 'Current_Ratio_Scaled': 0.4, 'Revenue_Growth_Scaled': 0.3},
            'Compliance': {'TCFD_Disclosure_Flag': 0.5, 'SEBI_BRSR_Flag': 0.5},
            'Climate': {'Climate_Risk_Scaled': 1.0}
        },
        'Retail': {
            'E': {'Scope1_GHG_Scaled': 0.2, 'Scope2_GHG_Scaled': 0.2, 'Water_Intensity_Scaled': 0.2, 'Waste_Diversion_Scaled': 0.4, 'Renewable_Energy_Scaled': 0.0}, # Supply chain, waste
            'S': {'Female_Leadership_Scaled': 0.3, 'Employee_Turnover_Scaled': 0.3, 'Community_Investment_Scaled': 0.2, 'Grievances_Resolved_Scaled': 0.2, 'News_Sentiment_Risk_Scaled': 0.0}, # Labor practices
            'G': {'Board_Independence_Scaled': 0.4, 'Audit_Committee_Scaled': 0.3, 'Corruption_Incidents_Scaled': 0.3},
            'Financial': {'Debt_Equity_Scaled': 0.3, 'Current_Ratio_Scaled': 0.4, 'Revenue_Growth_Scaled': 0.3},
            'Compliance': {'TCFD_Disclosure_Flag': 0.5, 'SEBI_BRSR_Flag': 0.5},
            'Climate': {'Climate_Risk_Scaled': 1.0}
        },
        # Add more industry types if you expand your dataset
        'Automotive': {
            'E': {'Scope1_GHG_Scaled': 0.3, 'Scope2_GHG_Scaled': 0.3, 'Water_Intensity_Scaled': 0.1, 'Waste_Diversion_Scaled': 0.1, 'Renewable_Energy_Scaled': 0.2},
            'S': {'Female_Leadership_Scaled': 0.2, 'Employee_Turnover_Scaled': 0.3, 'Community_Investment_Scaled': 0.2, 'Grievances_Resolved_Scaled': 0.2, 'News_Sentiment_Risk_Scaled': 0.1},
            'G': {'Board_Independence_Scaled': 0.4, 'Audit_Committee_Scaled': 0.3, 'Corruption_Incidents_Scaled': 0.3},
            'Financial': {'Debt_Equity_Scaled': 0.4, 'Current_Ratio_Scaled': 0.3, 'Revenue_Growth_Scaled': 0.3},
            'Compliance': {'TCFD_Disclosure_Flag': 0.6, 'SEBI_BRSR_Flag': 0.4},
            'Climate': {'Climate_Risk_Scaled': 1.0}
        },
        'Manufacturing': {
            'E': {'Scope1_GHG_Scaled': 0.35, 'Scope2_GHG_Scaled': 0.25, 'Water_Intensity_Scaled': 0.15, 'Waste_Diversion_Scaled': 0.15, 'Renewable_Energy_Scaled': 0.1},
            'S': {'Female_Leadership_Scaled': 0.2, 'Employee_Turnover_Scaled': 0.3, 'Community_Investment_Scaled': 0.2, 'Grievances_Resolved_Scaled': 0.2, 'News_Sentiment_Risk_Scaled': 0.1},
            'G': {'Board_Independence_Scaled': 0.4, 'Audit_Committee_Scaled': 0.3, 'Corruption_Incidents_Scaled': 0.3},
            'Financial': {'Debt_Equity_Scaled': 0.4, 'Current_Ratio_Scaled': 0.3, 'Revenue_Growth_Scaled': 0.3},
            'Compliance': {'TCFD_Disclosure_Flag': 0.5, 'SEBI_BRSR_Flag': 0.5},
            'Climate': {'Climate_Risk_Scaled': 1.0}
        },
        'Infrastructure': {
            'E': {'Scope1_GHG_Scaled': 0.3, 'Scope2_GHG_Scaled': 0.3, 'Water_Intensity_Scaled': 0.1, 'Waste_Diversion_Scaled': 0.1, 'Renewable_Energy_Scaled': 0.2},
            'S': {'Female_Leadership_Scaled': 0.1, 'Employee_Turnover_Scaled': 0.2, 'Community_Investment_Scaled': 0.4, 'Grievances_Resolved_Scaled': 0.2, 'News_Sentiment_Risk_Scaled': 0.1}, # Community impact high
            'G': {'Board_Independence_Scaled': 0.4, 'Audit_Committee_Scaled': 0.3, 'Corruption_Incidents_Scaled': 0.3},
            'Financial': {'Debt_Equity_Scaled': 0.4, 'Current_Ratio_Scaled': 0.3, 'Revenue_Growth_Scaled': 0.3},
            'Compliance': {'TCFD_Disclosure_Flag': 0.6, 'SEBI_BRSR_Flag': 0.4},
            'Climate': {'Climate_Risk_Scaled': 1.0}
        },
        'FMCG': {
            'E': {'Scope1_GHG_Scaled': 0.2, 'Scope2_GHG_Scaled': 0.2, 'Water_Intensity_Scaled': 0.3, 'Waste_Diversion_Scaled': 0.2, 'Renewable_Energy_Scaled': 0.1},
            'S': {'Female_Leadership_Scaled': 0.3, 'Employee_Turnover_Scaled': 0.2, 'Community_Investment_Scaled': 0.2, 'Grievances_Resolved_Scaled': 0.2, 'News_Sentiment_Risk_Scaled': 0.1},
            'G': {'Board_Independence_Scaled': 0.4, 'Audit_Committee_Scaled': 0.3, 'Corruption_Incidents_Scaled': 0.3},
            'Financial': {'Debt_Equity_Scaled': 0.3, 'Current_Ratio_Scaled': 0.4, 'Revenue_Growth_Scaled': 0.3},
            'Compliance': {'TCFD_Disclosure_Flag': 0.5, 'SEBI_BRSR_Flag': 0.5},
            'Climate': {'Climate_Risk_Scaled': 1.0}
        },
         'Telecom': {
            'E': {'Scope1_GHG_Scaled': 0.2, 'Scope2_GHG_Scaled': 0.2, 'Water_Intensity_Scaled': 0.2, 'Waste_Diversion_Scaled': 0.2, 'Renewable_Energy_Scaled': 0.2},
            'S': {'Female_Leadership_Scaled': 0.3, 'Employee_Turnover_Scaled': 0.2, 'Community_Investment_Scaled': 0.1, 'Grievances_Resolved_Scaled': 0.2, 'News_Sentiment_Risk_Scaled': 0.2},
            'G': {'Board_Independence_Scaled': 0.4, 'Audit_Committee_Scaled': 0.3, 'Corruption_Incidents_Scaled': 0.3},
            'Financial': {'Debt_Equity_Scaled': 0.3, 'Current_Ratio_Scaled': 0.4, 'Revenue_Growth_Scaled': 0.3},
            'Compliance': {'TCFD_Disclosure_Flag': 0.5, 'SEBI_BRSR_Flag': 0.5},
            'Climate': {'Climate_Risk_Scaled': 1.0}
        },
        # Default weights for any industry not explicitly listed
        'DEFAULT': {
            'E': {'Scope1_GHG_Scaled': 0.25, 'Scope2_GHG_Scaled': 0.25, 'Water_Intensity_Scaled': 0.15, 'Waste_Diversion_Scaled': 0.15, 'Renewable_Energy_Scaled': 0.2},
            'S': {'Female_Leadership_Scaled': 0.2, 'Employee_Turnover_Scaled': 0.2, 'Community_Investment_Scaled': 0.2, 'Grievances_Resolved_Scaled': 0.2, 'News_Sentiment_Risk_Scaled': 0.2},
            'G': {'Board_Independence_Scaled': 0.4, 'Audit_Committee_Scaled': 0.3, 'Corruption_Incidents_Scaled': 0.3},
            'Financial': {'Debt_Equity_Scaled': 0.35, 'Current_Ratio_Scaled': 0.35, 'Revenue_Growth_Scaled': 0.3},
            'Compliance': {'TCFD_Disclosure_Flag': 0.5, 'SEBI_BRSR_Flag': 0.5},
            'Climate': {'Climate_Risk_Scaled': 1.0}
        }
    }

    # Apply materiality-based scoring
    def calculate_pillar_score(row, pillar_type):
        industry_type = row['Industry']
        weights = industry_weights.get(industry_type, industry_weights['DEFAULT'])[pillar_type]
        score = 0
        for metric, weight in weights.items():
            score += row[metric] * weight
        return score

    df['E_Score'] = df.apply(lambda row: calculate_pillar_score(row, 'E'), axis=1)
    df['S_Score'] = df.apply(lambda row: calculate_pillar_score(row, 'S'), axis=1)
    df['G_Score'] = df.apply(lambda row: calculate_pillar_score(row, 'G'), axis=1)
    df['Financial_Risk_Score'] = df.apply(lambda row: calculate_pillar_score(row, 'Financial'), axis=1)

    # Compliance Risk Score (Non-compliance increases risk)
    df['Compliance_Risk_Score'] = df.apply(lambda row: (1 - row['TCFD_Disclosure_Flag']) * industry_weights.get(row['Industry'], industry_weights['DEFAULT'])['Compliance']['TCFD_Disclosure_Flag'] +
                                                      (1 - row['SEBI_BRSR_Flag']) * industry_weights.get(row['Industry'], industry_weights['DEFAULT'])['Compliance']['SEBI_BRSR_Flag'], axis=1)

    # Composite Risk Index (CRI) - Higher CRI = Higher Risk
    # This is the crucial fusion point
    cri_weights = { 
        'E_Score': 0.20,
        'S_Score': 0.20,
        'G_Score': 0.20,
        'Financial_Risk_Score': 0.20,
        'Compliance_Risk_Score': 0.10,
        'Climate_Risk_Scaled': 0.05, 
        'News_Sentiment_Controversy_Impact': 0.05 
    }

    # Factor in controversy directly into CRI for a more immediate impact
    # If Controversy_Flag is 1, add a penalty, otherwise 0
    df['News_Sentiment_Controversy_Impact'] = df['News_Sentiment_Risk_Scaled'] * 0.5 + df['Controversy_Flag'] * 0.5 
    df['News_Sentiment_Controversy_Impact'] = df['News_Sentiment_Controversy_Impact'].clip(0,1) 

    df['Composite_Risk_Index'] = (
        df['E_Score'] * cri_weights['E_Score'] +
        df['S_Score'] * cri_weights['S_Score'] +
        df['G_Score'] * cri_weights['G_Score'] +
        df['Financial_Risk_Score'] * cri_weights['Financial_Risk_Score'] +
        df['Compliance_Risk_Score'] * cri_weights['Compliance_Risk_Score'] +
        df['Climate_Risk_Scaled'] * cri_weights['Climate_Risk_Scaled'] +
        df['News_Sentiment_Controversy_Impact'] * cri_weights['News_Sentiment_Controversy_Impact']
    )


    df['Composite_Risk_Index'] = df['Composite_Risk_Index'] * 100

    # --- 4. Risk Level Classification ---
    # Define thresholds. These are critical and can be adjusted based on risk appetite.
    bins = [0, 50, 65, 101] 
    labels = ['Low', 'Medium', 'High'] 
    df['Risk_Level'] = pd.cut(df['Composite_Risk_Index'], bins=bins, labels=labels, include_lowest=True, right=False)
   
    # --- 5. Explainability Engine ---
    df['Risk_Explanation'] = ''
    for index, row in df.iterrows():
        explanation = [f"{row['Company']} is classified as {row['Risk_Level']} risk (CRI: {row['Composite_Risk_Index']:.2f})."]

        # High Risk reasons
        if row['Risk_Level'] == 'High':
            explanation.append("Key concerns include:")
            if row['Financial_Risk_Score'] > 0.7:
                explanation.append("  - Significant Financial Risk (e.g., high debt, low liquidity, poor revenue growth).")
            if row['E_Score'] > 0.7:
                explanation.append("  - High Environmental Risk (e.g., high GHG emissions, poor waste management).")
            if row['S_Score'] > 0.7:
                explanation.append("  - Elevated Social Risk (e.g., high employee turnover, low diversity).")
            if row['G_Score'] > 0.7:
                explanation.append("  - Weak Governance (e.g., low board independence, reported corruption).")
            if row['News_Sentiment_Controversy_Impact'] > 0.7 or row['Controversy_Flag'] == 1:
                explanation.append(f"  - Negative Reputational Risk due to recent adverse news ({row['Controversy_Detail'] if row['Controversy_Detail'] else 'general negative sentiment'}).")
            if row['Compliance_Risk_Score'] > 0.5:
                compliance_issues = []
                if row['SEBI_BRSR_Flag'] == 0: compliance_issues.append("SEBI BRSR non-compliance.")
                if row['TCFD_Disclosure_Flag'] == 0: compliance_issues.append("Lack of TCFD disclosure.")
                if compliance_issues: explanation.append(f"  - Regulatory non-compliance: {', '.join(compliance_issues)}")
            if row['Climate_Risk_Scaled'] > 0.7:
                explanation.append(f"  - High Physical Climate Risk based on location ({row['City_HQ']}).")

        # Medium Risk reasons
        elif row['Risk_Level'] == 'Medium':
            explanation.append("Identified areas for improvement:")
            if row['Financial_Risk_Score'] > 0.5:
                explanation.append("  - Moderate Financial Vulnerability.")
            if row['E_Score'] > 0.5:
                explanation.append("  - Moderate Environmental Performance.")
            if row['S_Score'] > 0.5:
                explanation.append("  - Moderate Social Issues.")
            if row['G_Score'] > 0.5:
                explanation.append("  - Moderate Governance Gaps.")
            if row['News_Sentiment_Controversy_Impact'] > 0.5:
                explanation.append("  - Some reputational concerns / neutral-to-negative sentiment.")
            if row['Compliance_Risk_Score'] > 0.2 and (row['SEBI_BRSR_Flag'] == 0 or row['TCFD_Disclosure_Flag'] == 0):
                explanation.append("  - Partial regulatory disclosure/compliance concerns.")
            if row['Climate_Risk_Scaled'] > 0.5:
                explanation.append(f"  - Moderate Physical Climate Risk ({row['City_HQ']}).")

        # Low Risk reasons
        else: # 'Low'
            explanation.append("Demonstrates strong performance across key risk dimensions:")
            explanation.append("  - Robust Financial Health.")
            explanation.append("  - Strong ESG Profile (Environmental, Social, Governance).")
            explanation.append("  - Positive or Neutral News Sentiment.")
            explanation.append("  - Good Regulatory Compliance.")
            explanation.append("  - Lower Exposure to Physical Climate Risks.")

        df.at[index, 'Risk_Explanation'] = "\n".join(explanation)

    # --- Clean up and Save ---
    # Drop intermediate scaled columns
    columns_to_drop_scaled = [col for col in df.columns if '_Scaled' in col]
    df.drop(columns=columns_to_drop_scaled, inplace=True)

    # Reorder columns for final output (add new scored columns explicitly)
    final_output_columns = [
        'Company', 'Industry', 'City_HQ', 'Latitude', 'Longitude', 'Simulated_Climate_Risk_Index',
        'Scope1_GHG_Emissions', 'Scope2_GHG_Emissions', 'Water_Consumption_Intensity', 'Waste_Diversion_Rate', 'Renewable_Energy_Percentage',
        'Female_Leadership_Ratio', 'Employee_Turnover_Rate', 'Community_Investment_PBT_Percentage', 'Grievances_Resolved_Percentage',
        'Board_Independence_Percentage', 'Audit_Committee_Independence_Percentage', 'Corruption_Incidents',
        'Debt_To_Equity', 'Current_Ratio', 'Revenue_Growth_Percentage',
        'News_Sentiment', 'Controversy_Flag', 'Controversy_Detail',
        'TCFD_Disclosure_Flag', 'SEBI_BRSR_Flag',
        'E_Score', 'S_Score', 'G_Score', 'Financial_Risk_Score', 'Compliance_Risk_Score', 'News_Sentiment_Controversy_Impact',
        'Composite_Risk_Index', 'Risk_Level', 'Risk_Explanation'
    ]
    df = df[final_output_columns]

    df.to_csv(output_csv, index=False)
    print(f"\n✅ ESG Risk Scoring complete. Output saved to '{output_csv}'.")
    print("\nFirst 5 rows of the scored dataset:")
    print(df.head())

# --- Main execution ---
if __name__ == "__main__":
    # Ensure ESG_Enhanced_Raw_Data.csv is generated by running generate_esg_dataset.py first
    run_advanced_esg_risk_engine()

Processing data from 'ESG_Enhanced_Raw_Data.csv'...
  - Imputed missing values in 'Scope1_GHG_Emissions' with median: 24053.07
  - Imputed missing values in 'Board_Independence_Percentage' with median: 70.81

✅ ESG Risk Scoring complete. Output saved to 'ESG_Risk_Scored_Output.csv'.

First 5 rows of the scored dataset:
               Company    Industry    City_HQ  Latitude  Longitude  \
0            HDFC Bank     Finance     Mumbai   19.0760    72.8777   
1  Reliance Industries      Energy     Mumbai   19.0760    72.8777   
2              Infosys  Technology  Bengaluru   12.9716    77.5946   
3          Tata Motors  Automotive       Pune   18.5204    73.8567   
4     Apollo Hospitals  Healthcare    Chennai   13.0827    80.2707   

   Simulated_Climate_Risk_Index  Scope1_GHG_Emissions  Scope2_GHG_Emissions  \
0                          7.25                955.64               1517.59   
1                          7.23              25523.15              80665.84   
2                    

In [16]:

def check_risk_counts(file_path='ESG_Risk_Scored_Output.csv'):
    """
    Reads the output CSV and prints the count of companies for each risk level.
    """

    # ---  Read the file safely ---
    try:
        df = pd.read_csv(file_path)
    except FileNotFoundError:
        print(f"Error: File '{file_path}' not found. Please ensure esg_risk_engine.py was run successfully.")
        return

    # --- Count companies per Risk_Level ---
    # Note: Ensure 'labels' in esg_risk_engine.py match exactly ('High', 'Medium', 'Low')
    risk_counts = df['Risk_Level'].value_counts()


    # --- Print Summary ---
    print(f"\n--- Risk Level Company Counts in '{file_path}' ---")
    print(f"Total Companies: {len(df)}")
    print(risk_counts) 

    
     # --- Print Individual Risk Category Counts ---
    print("\nIndividual Counts:")
    print(f"High Risk Companies: {risk_counts.get('High', 0)}")
    print(f"Medium Risk Companies: {risk_counts.get('Medium', 0)}")
    print(f"Low Risk Companies: {risk_counts.get('Low', 0)}")

# --- Run this function ---
if __name__ == "__main__":
    check_risk_counts()


--- Risk Level Company Counts in 'ESG_Risk_Scored_Output.csv' ---
Total Companies: 30
Risk_Level
Low       23
Medium     5
High       2
Name: count, dtype: int64

Individual Counts:
High Risk Companies: 2
Medium Risk Companies: 5
Low Risk Companies: 23


In [17]:
def get_cri_stats(file_path='ESG_Risk_Scored_Output.csv'):
    """
    Loads the scored CSV and prints statistics for Composite_Risk_Index.
    """

    # ---  Load the CSV file safely ---
    try:
        df = pd.read_csv(file_path)
    except FileNotFoundError:
        print(f"Error: File '{file_path}' not found. Please ensure esg_risk_engine.py was run successfully.")
        return


    # --- Compute CRI Statistics ---
    max_cri = df['Composite_Risk_Index'].max()
    min_cri = df['Composite_Risk_Index'].min()
    mean_cri = df['Composite_Risk_Index'].mean()

    # --- Print Results in Clear Format ---
    print(f"\n--- Composite_Risk_Index Statistics in '{file_path}' ---")
    print(f"Maximum CRI: {max_cri:.2f}")
    print(f"Minimum CRI: {min_cri:.2f}")
    print(f"Average CRI: {mean_cri:.2f}")

# --- Run this function ---
if __name__ == "__main__":
    get_cri_stats()


--- Composite_Risk_Index Statistics in 'ESG_Risk_Scored_Output.csv' ---
Maximum CRI: 68.38
Minimum CRI: 28.07
Average CRI: 44.85


In [18]:
# --- Define Function to Retrieve Companies by Risk Level ---
def get_companies_by_risk_level(file_path='ESG_Risk_Scored_Output.csv'):
    """
    Reads the output CSV and prints the names of companies for each risk level.
    """

    # --- Load the CSV file safely ---
    try:
        df = pd.read_csv(file_path)
    except FileNotFoundError:
        print(f"Error: File '{file_path}' not found. Please ensure esg_risk_engine.py was run successfully.")
        return

    print(f"\n--- Companies by Risk Level in '{file_path}' ---")

    # --- Define Risk Level Labels ---
    # Adjust these based on what was used in esg_risk_engine.py
    high_risk_label = 'High'
    medium_risk_label = 'Medium'
    low_risk_label = 'Low'

    # If you used plain text labels (e.g., ['Low', 'Medium', 'High']):
    # high_risk_label = 'High'
    # medium_risk_label = 'Medium'
    # low_risk_label = 'Low'


    # --- Extract and Print High Risk Companies ---
    high_risk_companies = df[df['Risk_Level'] == high_risk_label]['Company'].tolist()
    print(f"\n{high_risk_label} Companies ({len(high_risk_companies)}):")
    if high_risk_companies:
        for company in high_risk_companies:
            print(f"  - {company}")
    else:
        print("  None")

    # --- Extract and Print Medium Risk Companies ---
    medium_risk_companies = df[df['Risk_Level'] == medium_risk_label]['Company'].tolist()
    print(f"\n{medium_risk_label} Companies ({len(medium_risk_companies)}):")
    if medium_risk_companies:
        for company in medium_risk_companies:
            print(f"  - {company}")
    else:
        print("  None")

    # --- Extract and Print Low Risk Companies ---
    low_risk_companies = df[df['Risk_Level'] == low_risk_label]['Company'].tolist()
    print(f"\n{low_risk_label} Companies ({len(low_risk_companies)}):")
    if low_risk_companies:
        for company in low_risk_companies:
            print(f"  - {company}")
    else:
        print("  None")

# --- Run this function ---
if __name__ == "__main__":
    get_companies_by_risk_level()


--- Companies by Risk Level in 'ESG_Risk_Scored_Output.csv' ---

High Companies (2):
  - JSW Steel
  - Vedanta Ltd

Medium Companies (5):
  - DMart (Avenue Supermarts)
  - Power Grid Corp
  - Adani Ports
  - Indian Oil Corp
  - Zydus Lifesciences

Low Companies (23):
  - HDFC Bank
  - Reliance Industries
  - Infosys
  - Tata Motors
  - Apollo Hospitals
  - Asian Paints
  - L&T
  - Wipro
  - Maruti Suzuki
  - Godrej Consumer Products
  - Bharti Airtel
  - Nestle India
  - SBI
  - TCS
  - Cipla
  - IndusInd Bank
  - Aditya Birla Fashion
  - Dr. Reddy's Labs
  - HCL Tech
  - Bajaj Finance
  - United Spirits
  - Grasim Industries
  - Siemens India
